## Dataset

O nosso dataset providencia-nos dados geográficos e contagem de casos confirmados, óbitos e recuperados do Covid-19 num periodo de  4 meses (22 de janeiro de 2020 a 5 de maio de 2020) em vários países. O objetivo será extrair informação dos dados de forma a podermos construir um modelo de regressão capaz de prever as contagens tendo por base os fatores disponiveis (localização geográfica, dia e contagem anterior). Assim sendo é necessário partir dos dados iniciais e proceder a um pré-processamento para tratar os dados e poder-se então utlizar os algoritmos de regressão que pretendemos utilizar.

### Variáveis independentes

Como foi supramencionado, pretendemos utilizar certos fatores para fundamentarmos a nossa previsão (partindo dos dados do dataset). Estes então serão as coordenadas geográficas (Latitude e Longitude), número de dias desde o inicio da previsão (ou seja, 22 de janeiro) e contagens anteriores (número de casos confirmados, óbitos e recuperados do dia anterior).

### Variáveis Dependentes

Os valores que vamos prever serão os números de casos confirmados, óbitos e recuperados, sendo então estas as nossas variáveis independentes.

## Tratamento de dados

Para podermos obter resultados fiáveis, é preciso haver um tratamento dos dados que vem do dataset.
Pegando dos dados processados anteriormente feito inicialmente, começamos por extrai-los do ficheiro para podermos manipulá-los.

In [1]:
import pandas as pd

covid_data = pd.read_csv('covid_19_clean_complete.csv')
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27451,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27452,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27453,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27454,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


De seguida, após uma análise, verificou-se a presença de entrada respeitantes a navios que nalgum momento tiveram casos de Covid-19 e não estão portanto associados a nenhum país particular.
Além disso, consideramos que estes dados iriam criar ruído, pelo que optamos por ignorá-los e remover dos dados em análise.

In [2]:
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Grand Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='MS Zaandam'].index)
covid_data = covid_data.reset_index()
del covid_data['index']
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27035,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27036,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27037,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27038,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


A maioria das entradas da coluna **Province/State** tem valores nulos, pelo que procedemos a eliminá-los. Além disso, como ter uma entrada para uma região e país não é muito relevante, optamos por agregar as duas informações numa só coluna denominada de **Local**.

In [3]:
import numpy as np

covid_data['Province/State'] = covid_data.replace(np.nan, '', regex=True)
cols = ['Province/State', 'Country/Region']
covid_data['Local'] = covid_data[cols].apply(lambda row: ' / '.join(row.values.astype(str)) if row.values[0] != '' else ''.join(row.values.astype(str)), axis=1)
del covid_data['Province/State']
del covid_data['Country/Region']
covid_data

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,1/22/20,0,0,0,Afghanistan
1,41.153300,20.168300,1/22/20,0,0,0,Albania
2,28.033900,1.659600,1/22/20,0,0,0,Algeria
3,42.506300,1.521800,1/22/20,0,0,0,Andorra
4,-11.202700,17.873900,1/22/20,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,5/4/20,6,0,5,Western Sahara
27036,0.186360,6.613081,5/4/20,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,5/4/20,12,2,0,Yemen
27038,-11.645500,43.333300,5/4/20,3,0,0,Comoros


De seguida, vamos converter as datas em contagem de dias desde o início do dataset (22 de janeiro de 2020)

In [4]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'],format='%m/%d/%y')
covid_data['Date'] -= pd.to_datetime("2020-01-22")
covid_data['Date'] /= np.timedelta64(1,'D')
covid_data = covid_data.rename(columns  = {'Date':'Days Passed'})
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,0.0,0,0,0,Afghanistan
1,41.153300,20.168300,0.0,0,0,0,Albania
2,28.033900,1.659600,0.0,0,0,0,Algeria
3,42.506300,1.521800,0.0,0,0,0,Andorra
4,-11.202700,17.873900,0.0,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,103.0,12,2,0,Yemen
27038,-11.645500,43.333300,103.0,3,0,0,Comoros


Por fim, adicionar as colunas da contagem do dia anterior. Este passo é um pouco mais longo tendo em conta o numero de linhas e a procura pelo valor anterior.

In [5]:
covid_data['Conf. Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Confirmed'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data['Deaths Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Deaths'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data['Recov. Prev.'] = covid_data.apply(lambda row: 
                                                      covid_data[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1)]['Recovered'].item()
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local,Conf. Prev.,Deaths Prev.,Recov. Prev.
0,33.000000,65.000000,0.0,0,0,0,Afghanistan,0,0,0
1,41.153300,20.168300,0.0,0,0,0,Albania,0,0,0
2,28.033900,1.659600,0.0,0,0,0,Algeria,0,0,0
3,42.506300,1.521800,0.0,0,0,0,Andorra,0,0,0
4,-11.202700,17.873900,0.0,0,0,0,Angola,0,0,0
...,...,...,...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara,6,0,5
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe,16,1,4
27037,15.552727,48.516388,103.0,12,2,0,Yemen,10,2,0
27038,-11.645500,43.333300,103.0,3,0,0,Comoros,3,0,0


In [6]:
with pd.ExcelWriter('covid19_dataset_final.xlsx') as writer:
    covid_data.to_excel(writer)

## K Nearest Neighbours
Este dataset que processamos permite-nos agrupar certos fatores relevantes como a posição geográfica e contagem dos dias anteriores para  poder-se relacionar certos pontos (entradas de dados) e obter uma previsão dos dados tendo em conta aquelas que estão mais próximas, que é em que se baseia o algoritmo **K Nearest neighbours**.
Já que pretendemos construir um modelo de regressão destes dados, vamos recorrer à ferramenta *scikit-learn* para utilizar os seus algoritmos de regressão, nos quais se encontra o **KNeighborsRegressor**, que será o escolhido para aplicar o KNN.

### KNeighborsRegressor
#### Parâmetros da pesquisa
* **n_neighbors**: número de vizinhos para usar. Para efeitos de testes serão testados os valores:
    * *1*, *3*, *7*, *10*, *15*
* **weights**: função de peso utilizado na previsão. Opções a testar:
    * *uniform*: pesos uniformes, todos os pontos na vizinhança são pesados igualmente
    * *distance*: pesos influenciados pela distância ao ponto de pesquisa, em que pontos vizinhos mais pertos do ponto de pesquisa terão mais influência do que aqueles mais afastados
* **algorithm**: algoritmo usado para a função de **fit**. Opções a usar:
    * *kd_tree*, *ball_tree*, *brute*
* **leaf_size**: tamanho uma folha a ser usada pela função de fit com árvore (KDTree e BallTree)
    * *10*, *30*: default, *70*, *150*, *300*
* **metric**: função de métrica da distância, a usar com os pesos baseados na distância. Opções a testar:
    * *euclidean*, *chebyshev*, *manhattan* 
* **n_jobs**: definir nº de processos para paralelizar os trabalhos (otimização do tempo de pesquisa, não influencia a qualidade dos resultados obtidos)
    * *None*: não há paralelismo (usar em debug)
    * *-1*: todos os cpu's são usados
    
#### Criação dos sets de treino e teste
Procedemos então à criação de sets para podermos treinar o nosso modelo, e por fim testá-lo. Antes disso, escalamos os valores para ficarem normalizados.

In [7]:
from sklearn.model_selection import train_test_split

#colunas em que vamos basear as previsões
x_columns = ['Lat','Long','Days Passed', 'Conf. Prev.','Deaths Prev.','Recov. Prev.']
#colunas que queremos prever
y_columns = ['Confirmed','Deaths','Recovered']

#criar set de treino e teste
X_train, X_test, y_train, y_test = train_test_split(covid_data[x_columns], covid_data[y_columns], test_size=0.0096)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
unscaled_test = X_test.join(y_test)
X_test = scaler.transform(X_test)

#### Execução
Uma vez feita a preparação dos dados, avançamos então para a execução. Esta primeira será feita com os parâmetros por defeito, pelo de seguida passamos a exploração dos resultados não só com estes parâmetros, mas também com os outros acima mencionados.

Os parâmetros desta primeira execução são:
* *n_neighbors* = 5
* *weights* = 'uniform'
* *algorithm* = 'auto'
* *leaf_size* = 30
* *metric* = 'euclidean' (porque o default é minkowski como parâmetro p=2, resultando na distância euclideana)


In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import *

def runRegressor(params) :
    
    if(len(params) == 4):
        #criar o modelo e usar o número de vizinhos default, 5.
        knn = KNeighborsRegressor(n_neighbors=params[0],weights=params[1],algorithm=params[2],leaf_size=params[3],n_jobs=-1)
    else:
        #criar o modelo e usar o número de vizinhos default, 5.
        knn = KNeighborsRegressor(n_neighbors=params[0],weights=params[1],metric=params[2],algorithm=params[3],leaf_size=params[4],n_jobs=-1)
    #aplicar a função de fit ao set de treino
    knn.fit(X_train,y_train)
    #fazer previsões do set de teste usando 
    predictions = knn.predict(X_test)

    #tratar dos resultados
    predictions = pd.DataFrame(data=predictions,columns=['Confirmed Prediction','Deaths Prediction','Recovered Prediction'])

    #buscar os locais para apresentar as previsoes, ja que nao foram usados no modelo de regressão
    predictions['Local'] = unscaled_test.apply(lambda row: covid_data.loc[(covid_data['Lat'] == row['Lat']) & (covid_data['Long'] == row['Long']),'Local'].iloc[0],axis=1).tolist()

    #arredondar os valores para inteiros, com teto
    predictions['Confirmed Prediction'] = predictions['Confirmed Prediction'].apply(np.ceil)
    predictions['Deaths Prediction'] = predictions['Deaths Prediction'].apply(np.ceil)
    predictions['Recovered Prediction'] = predictions['Recovered Prediction'].apply(np.ceil)

    #comparar os valores previstos e reais guardar a diferença nas contagens
    predictions['Confirmed Diff'] = (unscaled_test['Confirmed'] - predictions['Confirmed Prediction'].values).tolist()
    predictions['Deaths Diff'] = (unscaled_test['Deaths'] - predictions['Deaths Prediction'].values).tolist()
    predictions['Recovered Diff'] = (unscaled_test['Recovered'] - predictions['Recovered Prediction'].values).tolist()
    #colocar o numero de dias passados para ser mais facil analisar
    predictions['Days Passed'] = unscaled_test['Days Passed'].tolist()

    predictions = predictions[['Days Passed','Local', 'Confirmed Prediction', 'Confirmed Diff','Deaths Prediction', 'Deaths Diff','Recovered Prediction','Recovered Diff']]
    
    y_true_conf = unscaled_test['Confirmed'].values
    y_true_deaths = unscaled_test['Deaths'].values
    y_true_recov = unscaled_test['Recovered'].values
    
    y_pred_conf = predictions['Confirmed Prediction'].values
    y_pred_deaths = predictions['Deaths Prediction'].values
    y_pred_recov = predictions['Recovered Prediction'].values

    #calcular o erro quadrático médio
    conf_mse = mean_squared_error(y_true_conf,y_pred_conf)
    deaths_mse = mean_squared_error(y_true_deaths,y_pred_deaths)
    recov_mse = mean_squared_error(y_true_recov,y_pred_recov)
    mse = {"conf_mse" : conf_mse,"deaths_mse": deaths_mse,"recov_mse":recov_mse}
    
    #calcular o erro absoluto médio
    conf_mae = mean_absolute_error(y_true_conf,y_pred_conf)
    deaths_mae = mean_absolute_error(y_true_deaths,y_pred_deaths)
    recov_mae = mean_absolute_error(y_true_recov,y_pred_recov)
    mae = {"conf_mae" : conf_mae,"deaths_mae": deaths_mae,"recov_mae":recov_mae}

    #calcular o  coeficiente R^2 de regressão 
    conf_r2 = r2_score(y_true_conf,y_pred_conf)
    deaths_r2 = r2_score(y_true_deaths,y_pred_deaths)
    recov_r2 = r2_score(y_true_recov,y_pred_recov)
    r2 = {"conf_r2" : conf_r2,"deaths_r2": deaths_r2,"recov_r2":recov_r2}
    
    results = {"params": params, "data": predictions,"mse" : mse, "mae": mae,"r2":r2}
    return results

#correr o default 
runRegressor((1,'uniform','auto',30))


{'params': (1, 'uniform', 'auto', 30),
 'data':      Days Passed                           Local  Confirmed Prediction  \
 0           36.0                        Djibouti                   0.0   
 1           15.0  Northwest Territories / Canada                   0.0   
 2           23.0    Isle of Man / United Kingdom                   0.0   
 3          100.0                 Tianjin / China                 328.0   
 4           12.0                           Malta                   0.0   
 ..           ...                             ...                   ...   
 255         41.0                      Madagascar                   0.0   
 256         19.0                      Mozambique                   0.0   
 257          7.0                            Chad                   0.0   
 258         50.0                    Burkina Faso                   2.0   
 259         40.0                            Chad                   0.0   
 
      Confirmed Diff  Deaths Prediction  Deaths Dif

In [9]:
n_neighbors = [1,3,7,10,15]
uni_wgts = ['uniform']
dist_wgts = ['distance']
metric = ['euclidean','manhattan','chebyshev']
bf_algorithm = ['brute']
tree_algorithm = ['kd_tree','ball_tree']
leaf_size = [10,30,70,150,300]

import itertools

uniform_tree_params = {"n_neighbors" : n_neighbors, "weigths" : uni_wgts, "algorithm": tree_algorithm, "leaf_size" : leaf_size}
uniform_bf_params = {"n_neighbors" : n_neighbors, "weigths" : uni_wgts, "algorithm": bf_algorithm, "leaf_size" : leaf_size}
dist_tree_params = {"n_neighbors" : n_neighbors, "weigths" : dist_wgts, "metric": metric, "algorithm": tree_algorithm, "leaf_size" : leaf_size}
dist_bf_params = {"n_neighbors" : n_neighbors, "weigths" : dist_wgts, "metric": metric, "algorithm": bf_algorithm, "leaf_size" : leaf_size}

combinations = [] 
for p in itertools.product(*uniform_tree_params.values()):
    combinations.append(p)
for p in itertools.product(*uniform_bf_params.values()):
    combinations.append(p)
for p in itertools.product(*dist_tree_params.values()):
    combinations.append(p)
for p in itertools.product(*dist_bf_params.values()):
    combinations.append(p)
 
combinations

[(1, 'uniform', 'kd_tree', 10),
 (1, 'uniform', 'kd_tree', 30),
 (1, 'uniform', 'kd_tree', 70),
 (1, 'uniform', 'kd_tree', 150),
 (1, 'uniform', 'kd_tree', 300),
 (1, 'uniform', 'ball_tree', 10),
 (1, 'uniform', 'ball_tree', 30),
 (1, 'uniform', 'ball_tree', 70),
 (1, 'uniform', 'ball_tree', 150),
 (1, 'uniform', 'ball_tree', 300),
 (3, 'uniform', 'kd_tree', 10),
 (3, 'uniform', 'kd_tree', 30),
 (3, 'uniform', 'kd_tree', 70),
 (3, 'uniform', 'kd_tree', 150),
 (3, 'uniform', 'kd_tree', 300),
 (3, 'uniform', 'ball_tree', 10),
 (3, 'uniform', 'ball_tree', 30),
 (3, 'uniform', 'ball_tree', 70),
 (3, 'uniform', 'ball_tree', 150),
 (3, 'uniform', 'ball_tree', 300),
 (7, 'uniform', 'kd_tree', 10),
 (7, 'uniform', 'kd_tree', 30),
 (7, 'uniform', 'kd_tree', 70),
 (7, 'uniform', 'kd_tree', 150),
 (7, 'uniform', 'kd_tree', 300),
 (7, 'uniform', 'ball_tree', 10),
 (7, 'uniform', 'ball_tree', 30),
 (7, 'uniform', 'ball_tree', 70),
 (7, 'uniform', 'ball_tree', 150),
 (7, 'uniform', 'ball_tree', 300)

In [10]:
import time

start_time = time.time()
results = []
for params in combinations:
    results.append(runRegressor(params))

print("Time elapsed for all combinations: %s seconds" % round((time.time()-start_time),4))
results

Time elapsed for all combinations: 71.8314 seconds


[{'params': (1, 'uniform', 'kd_tree', 10),
  'data':      Days Passed                           Local  Confirmed Prediction  \
  0           36.0                        Djibouti                   0.0   
  1           15.0  Northwest Territories / Canada                   0.0   
  2           23.0    Isle of Man / United Kingdom                   0.0   
  3          100.0                 Tianjin / China                 328.0   
  4           12.0                           Malta                   0.0   
  ..           ...                             ...                   ...   
  255         41.0                      Madagascar                   0.0   
  256         19.0                      Mozambique                   0.0   
  257          7.0                            Chad                   0.0   
  258         50.0                    Burkina Faso                   2.0   
  259         40.0                            Chad                   0.0   
  
       Confirmed Diff  Deaths Pred